In [1]:
import csv
import json

In [2]:
def parseRow(row, headers, datatypes):
    if len(row) == 0:
        return None
    
    result = {}
    for i, element in enumerate(row):
        dtype = datatypes[i]
        header = headers[i]
        obj = ""
        
        if len(header) == 0:
            continue
            
        ### Handle the following 'data types':
        #   unique, string, category, number, list_custom1, list, map
        if dtype == "unique":
            obj = element
        elif dtype == "string":
            print("Test:", headers[i], element)
            if len(element) == 0:
                obj = ""
            else:
                obj = element
        elif "category" in dtype:
            obj = element
        elif dtype == "number":
            obj = int(element)
        elif dtype == "list_custom1":
            obj = []
            if element:
                for item in element.split(';;'):
                    d = {}
                    for pair in item.split(';'):
                        item, value = pair.split(":")
                        d[item.strip()] = value.strip()
                    obj.append(d)
        elif "list" in dtype:
            obj = []
            if element:
                for item in element.split(';'):
                    obj.append(item.strip())
        elif "map" in dtype:
            obj = {}
            if element:
                for pair in element.split(";"):
                    item, value = pair.split(":")
                    obj[item.strip()] = value.strip()
                
        result[header] = obj
        
    return result

In [40]:
# JSON file will be created here
jsonPath = "../data/BedBugProductData.json"

# Source product data file has one row per product
csvPath = "../data/BedBugProductData.csv"


# Source ORPA data file has multiple lines per product, not all products have rows
# 'ORPA' stands for OtherReferencedProductAttributes
orpaPath = "../data/OtherReferencedProductAttributes.csv"
orpaData = {} # map product id -> list of products' rows in ORPA file

# Read and parse the ORPA data
with open(orpaPath, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=",", quotechar='"')
    header = next(reader)
    
    # read in all data
    productOrpaData = []
    for row in reader:
        productOrpaData.append(row)
    # sort by product id (column 0)
    productOrpaData.sort(key=lambda row: row[0])
    
    # reduce data down to 1 row per product
    currId = -1
    productOrpaList = []
    for row in productOrpaData:
        row_d = {header[i]: row[i] for i in range(len(header))}
        nextId = row_d.pop("id")
        if currId == -1:
            currId = nextId
        if currId == nextId:
            productOrpaList.append(row_d)
        else:
            orpaData[currId] = productOrpaList
            productOrpaList = [row_d]
            currId = nextId
    orpaData[currId] = productOrpaList

# Read, parse, and store the remaining product data as JSON file
with open(jsonPath, 'w') as jsonfile:
    with open(csvPath, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=",", quotechar='"')
        
        datatypes = next(reader)
        headers = next(reader)
        data = []
        
        for row in reader:
            parsedRow = parseRow(row, headers, datatypes)
            if parsedRow is not None:
                productId = parsedRow["id"]
                productOrpa = orpaData.get(productId)
                if productOrpa:
                    parsedRow["otherReferencedProductAttributes"] = productOrpa
                    
                data.append(parsedRow)
        
        json.dump(data, jsonfile)

print("Conversion completed. Refresh browser page to see changes.")

Test: labelDate 8/4/16
Test: reference 1
Test: labelDate 8/4/16
Test: reference 2
Test: labelDate 8/1/16
Test: reference 3
Test: labelDate 10/14/16
Test: reference 4
Test: labelDate none
Test: reference 5
Test: labelDate 2014
Test: reference 6
Test: labelDate 2014
Test: reference 7
Test: labelDate none
Test: reference 8
Test: labelDate none
Test: reference 9
Test: labelDate none
Test: reference 10
Test: labelDate 2013
Test: reference 11
Test: labelDate 7/29/14
Test: reference 12
Test: labelDate 2016
Test: reference 13
Test: labelDate 2015
Test: reference 14
Test: labelDate 2016
Test: reference 15
Test: labelDate 2016
Test: reference 16
Test: labelDate 2016
Test: reference 17
Test: labelDate 2015
Test: reference 18
Test: labelDate 2016
Test: reference 19
Test: labelDate 2013
Test: reference 20
Test: labelDate 2014
Test: reference 21
Test: labelDate 2014
Test: reference 22
Test: labelDate none
Test: reference 23
Test: labelDate 2008
Test: reference 24
Test: labelDate 8 Jul, 2015
Test: re